In [1]:
import os
import sys
import re
import time
import random
import warnings
import collections
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

sys.path.append('../../src')
import cb_utils

sns.set(style="darkgrid")
pd.options.display.max_columns = 500

%load_ext autoreload
%autoreload 2

# Service Fund etl one off

### Config

In [2]:
data_dir = '/Users/bp/workspace/msh/service_fund_data'
current_folder = '20240904'
# pro_file_name = 'ACO Builder MSSP Prospective Explorer v2025.3.txt'
# retro_file_name = 'ACO Builder MSSP Retrospective Explorer v2025.3.txt'
table_prefix = 'sf_data'

In [11]:
files = [f for f in os.listdir(f'{data_dir}/{current_folder}') if f != '.DS_Store']
files

['TX_TX0185252_MCR_TCOC_RX_CLAIM_FILE_2024_20240814_331.csv',
 'AR_AR0185420_MCR_TCOC_CAPITATION_FILE_2024_20240814_332.csv',
 'AR_AR0185420_MCR_TCOC_RX_CLAIM_FILE_2024_20240814_332.csv',
 'AR_AR0185420_MCR_TCOC_MED_CLAIMS_FILE_2024_20240814_332.csv',
 'TX_TX0185252_MCR_TCOC_MED_CLAIMS_FILE_2024_20240814_331.csv',
 'TX_TX0185252_MCR_TCOC_MEMBER_ELIG_MMR_FILE_2024_20240814_331.csv',
 'TX_TX0185252_MCR_TCOC_CAPITATION_FILE_2024_20240814_331.csv',
 'AR_AR0185420_MCR_TCOC_OTHER_ADJUSTMENTS_FILE_2023_20240814_332.csv',
 'AR_AR0185420_MCR_TCOC_MEMBER_ELIG_MMR_FILE_2024_20240814_332.csv',
 'TX_TX0185252_MCR_TCOC_OTHER_ADJUSTMENTS_FILE_2024_20240814_331.csv',
 'TX_TX0185252_MCR_TCOC_CAPITATION_FILE_2023_20240814_331.csv',
 'AR_AR0185420_MCR_TCOC_OTHER_ADJUSTMENTS_FILE_2024_20240814_332.csv',
 'TX_TX0185252_MCR_TCOC_MEMBER_ELIG_MMR_FILE_2023_20240814_331.csv',
 'AR_AR0185420_MCR_TCOC_MEMBER_ELIG_MMR_FILE_2023_20240814_332.csv',
 'TX_TX0185252_MCR_TCOC_OTHER_ADJUSTMENTS_FILE_2023_20240814_331.cs

In [57]:
'MCR_TCOC_' + 'MCR_TCOC_20240814_AR_AR0185420_OTHER_ADJUSTMENTS_FILE_2023_20240814_332'.replace('MCR_TCOC_', '').replace('.csv', '').replace('FILE_', '').lower()

'MCR_TCOC_20240814_ar_ar0185420_other_adjustments_2023_20240814_332'

In [59]:
files[9:]

['TX_TX0185252_MCR_TCOC_OTHER_ADJUSTMENTS_FILE_2024_20240814_331.csv',
 'TX_TX0185252_MCR_TCOC_CAPITATION_FILE_2023_20240814_331.csv',
 'AR_AR0185420_MCR_TCOC_OTHER_ADJUSTMENTS_FILE_2024_20240814_332.csv',
 'TX_TX0185252_MCR_TCOC_MEMBER_ELIG_MMR_FILE_2023_20240814_331.csv',
 'AR_AR0185420_MCR_TCOC_MEMBER_ELIG_MMR_FILE_2023_20240814_332.csv',
 'TX_TX0185252_MCR_TCOC_OTHER_ADJUSTMENTS_FILE_2023_20240814_331.csv',
 'TX_TX0185252_MCR_TCOC_MED_CLAIMS_FILE_2023_20240814_331.csv',
 'AR_AR0185420_MCR_TCOC_MED_CLAIMS_FILE_2023_20240814_332.csv',
 'TX_TX0185252_MCR_TCOC_RX_CLAIM_FILE_2023_20240814_331.csv',
 'AR_AR0185420_MCR_TCOC_RX_CLAIM_FILE_2023_20240814_332.csv',
 'AR_AR0185420_MCR_TCOC_CAPITATION_FILE_2023_20240814_332.csv']

In [60]:
for f in files[10:]:
    df = pd.read_csv(f'{data_dir}/{current_folder}/{f}', dtype=str)
    df = cb_utils.df_format_columns(df) 
    table_name = ('MCR_TCOC_20240814_' + re.sub(r'_FILE_202._20240814_.*', '', f.replace('MCR_TCOC_', ''))).lower()
    schema = 'raw'
    print(f'select * from {schema}.{table_name}')
    conn = cb_utils.get_engine(source='msh_analytics')
    df.to_sql(table_name, conn, schema=schema, index=False, if_exists='append', method='multi', chunksize=1000) # , if_exists='replace'

select * from raw.mcr_tcoc_20240814_tx_tx0185252_capitation
select * from raw.mcr_tcoc_20240814_ar_ar0185420_other_adjustments
select * from raw.mcr_tcoc_20240814_tx_tx0185252_member_elig_mmr
select * from raw.mcr_tcoc_20240814_ar_ar0185420_member_elig_mmr
select * from raw.mcr_tcoc_20240814_tx_tx0185252_other_adjustments
select * from raw.mcr_tcoc_20240814_tx_tx0185252_med_claims
select * from raw.mcr_tcoc_20240814_ar_ar0185420_med_claims
select * from raw.mcr_tcoc_20240814_tx_tx0185252_rx_claim
select * from raw.mcr_tcoc_20240814_ar_ar0185420_rx_claim
select * from raw.mcr_tcoc_20240814_ar_ar0185420_capitation
